In [ ]:
import argparse
import csv
import re
import requests
from collections import Counter
from datetime import datetime

def download_file(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text.splitlines()

def process_csv(file_lines):
    reader = csv.reader(file_lines)
    data = [row for row in reader]
    return data

def count_image_hits(data):
    image_pattern = re.compile(r'.*\.(jpg|png|gif)$', re.IGNORECASE)
    total_hits = len(data)
    image_hits = sum(1 for row in data if image_pattern.match(row[0]))
    image_percentage = (image_hits / total_hits) * 100 if total_hits > 0 else 0
    print(f"Image requests account for {image_percentage:.2f}% of all requests")

def find_popular_browser(data):
    browser_patterns = {
        'Firefox': re.compile(r'Firefox', re.IGNORECASE),
        'Chrome': re.compile(r'Chrome', re.IGNORECASE),
        'Internet Explorer': re.compile(r'MSIE|Trident', re.IGNORECASE),
        'Safari': re.compile(r'Safari', re.IGNORECASE)
    }
    browser_counts = Counter()
    for row in data:
        user_agent = row[2]
        for browser, pattern in browser_patterns.items():
            if pattern.search(user_agent):
                browser_counts[browser] += 1
                break
    most_common_browser = browser_counts.most_common(1)
    if most_common_browser:
        print(f"Most popular browser: {most_common_browser[0][0]}")

def count_hits_by_hour(data):
    hour_counts = Counter()
    for row in data:
        dt = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")
        hour_counts[dt.hour] += 1
    for hour, count in sorted(hour_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"Hour {hour:02d} has {count} hits")

def main():
    url = "http://s3.amazonaws.com/cuny-is211-spring2015/weblog.csv"
    file_lines = download_file(url)
    data = process_csv(file_lines)
    count_image_hits(data)
    find_popular_browser(data)
    count_hits_by_hour(data)

if __name__ == "__main__":
    main()
